<a href="https://colab.research.google.com/github/tasmita0131/06_mainfile/blob/main/EE488_tasmita_tanjim_19723_assignment06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
def alu(a, b, opcode):
    # Ensure 8-bit inputs
    A = a & 0xFF
    B = b & 0xFF

    if opcode == '0000':      # A + B
        return (A + B) & 0xFF
    elif opcode == '0001':    # A - B
        return (A - B) & 0xFF
    elif opcode == '0010':    # A * B
        return (A * B) & 0xFF
    elif opcode == '0011':    # A / B
        return (A // B) if B != 0 else 0
    elif opcode == '0100':    # A << 1
        return (A << 1) & 0xFF
    elif opcode == '0101':    # A >> 1
        return A >> 1
    elif opcode == '0110':    # Rotate A left by 1
        return ((A << 1) | (A >> 7)) & 0xFF
    elif opcode == '0111':    # Rotate A right by 1
        return ((A >> 1) | ((A & 1) << 7)) & 0xFF
    elif opcode == '1000':    # A AND B
        return A & B
    elif opcode == '1001':    # A OR B
        return A | B
    elif opcode == '1010':    # A XOR B
        return A ^ B
    elif opcode == '1011':    # A NOR B
        return ~(A | B) & 0xFF
    elif opcode == '1100':    # A NAND B
        return ~(A & B) & 0xFF
    elif opcode == '1101':    # A XNOR B
        return ~(A ^ B) & 0xFF
    elif opcode == '1110':    # A > B
        return 1 if A > B else 0
    elif opcode == '1111':    # A == B
        return 1 if A == B else 0
    else:
        return None  # Invalid opcode

# Test examples with compact output
print("Add 5+3:", alu(5, 3, '0000'))
print("Add 255+1 (overflow):", alu(255, 1, '0000'))
print("Sub 8-2:", alu(8, 2, '0001'))
print("Mul 5*3:", alu(5, 3, '0010'))
print("Div 9/3:", alu(9, 3, '0011'))
print("Left shift 5<<1:", alu(5, 0, '0100'))
print("Right shift 8>>1:", alu(8, 0, '0101'))
print("Rotate left 128:", alu(128, 0, '0110'))
print("Rotate right 1:", alu(1, 0, '0111'))
print("AND 12&10:", alu(12, 10, '1000'))
print("OR 12|10:", alu(12, 10, '1001'))
print("XOR 12^10:", alu(12, 10, '1010'))
print("NOR 12 nor 10:", alu(12, 10, '1011'))
print("NAND 12 nand 10:", alu(12, 10, '1100'))
print("XNOR 12 xnor 10:", alu(12, 10, '1101'))
print("Compare 7>5:", alu(7, 5, '1110'))
print("Compare 10==10:", alu(10, 10, '1111'))

Add 5+3: 8
Add 255+1 (overflow): 0
Sub 8-2: 6
Mul 5*3: 15
Div 9/3: 3
Left shift 5<<1: 10
Right shift 8>>1: 4
Rotate left 128: 1
Rotate right 1: 128
AND 12&10: 8
OR 12|10: 14
XOR 12^10: 6
NOR 12 nor 10: 241
NAND 12 nand 10: 247
XNOR 12 xnor 10: 249
Compare 7>5: 1
Compare 10==10: 1


In [12]:
def arithmetic_right_shift(A, Q, Q1):
    combined = A + Q + [Q1]
    shifted = [combined[0]] + combined[:-1]
    return shifted[:4], shifted[4:8], shifted[8]

def add_binary(A, B):
    result = [0] * len(A)
    carry = 0
    for i in range(len(A) - 1, -1, -1):
        total = A[i] + B[i] + carry
        result[i] = total % 2
        carry = total // 2
    return result[-4:]  # Keep only last 4 bits

def twos_complement(bin_list):
    inverted = [1 - bit for bit in bin_list]
    one = [0] * (len(bin_list) - 1) + [1]
    return add_binary(inverted, one)

def booth_multiplier(multiplicand, multiplier):
    A = [0, 0, 0, 0]
    Q = [int(x) for x in f"{abs(multiplier):04b}"]
    M = [int(x) for x in f"{abs(multiplicand):04b}"]
    Q1 = 0
    count = 4

    if multiplicand < 0:
        M = twos_complement(M)
    if multiplier < 0:
        Q = twos_complement(Q)

    while count > 0:
        if Q[-1] == 1 and Q1 == 0:
            A = add_binary(A, twos_complement(M))  # A = A - M
        elif Q[-1] == 0 and Q1 == 1:
            A = add_binary(A, M)  # A = A + M
        A, Q, Q1 = arithmetic_right_shift(A, Q, Q1)
        count -= 1

    result = A + Q
    value = int("".join(map(str, result)), 2)
    if result[0] == 1:
        value -= (1 << 8)
    return value


def shift_add_multiplier(multiplicand, multiplier):
    product = 0
    negative_result = (multiplicand < 0) ^ (multiplier < 0)
    multiplicand = abs(multiplicand)
    multiplier = abs(multiplier)

    for i in range(4):
        if (multiplier >> i) & 1:
            product += (multiplicand << i)

    if negative_result:
        product = -product
    return product & 0xFF  # Limit to 8-bit result


if __name__ == "__main__":
    print("Booth’s Algorithm Results:")
    print(" -3 * 2 =", booth_multiplier(-3, 2))
    print(" -4 * -2 =", booth_multiplier(-4, -2))
    print(" 5 * -3 =", booth_multiplier(5, -3))

    print("\nShift-and-Add Results:")
    print(" 3 * 2 =", shift_add_multiplier(3, 2))
    print(" 7 * 4 =", shift_add_multiplier(7, 4))
    print(" 15 * 15 =", shift_add_multiplier(15, 15))


Booth’s Algorithm Results:
 -3 * 2 = -6
 -4 * -2 = 8
 5 * -3 = -15

Shift-and-Add Results:
 3 * 2 = 6
 7 * 4 = 28
 15 * 15 = 225
